In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

In [4]:
print(os.getcwd())

C:\Users\harol\Downloads


In [2]:
import numpy as np
import time
import pandas as pd

In [3]:
driver = webdriver.Chrome()

In [4]:
df_player = pd.read_csv('df_player.csv')
df_player.drop(columns = ['Unnamed: 0'], inplace = True)
df_player

,Player,Team,Number,Position,Height,Weight,Last Attended,Country
0,PreciousAchiuwa,TOR,5.0,F,6-8,225 lbs,Memphis,Nigeria
1,StevenAdams,MEM,4.0,C,6-11,265 lbs,Pittsburgh,New Zealand
2,BamAdebayo,MIA,13.0,C-F,6-9,255 lbs,Kentucky,USA
3,SantiAldama,MEM,7.0,F-C,6-11,215 lbs,Loyola-Maryland,Spain
4,LaMarcusAldridge,BKN,21.0,C-F,6-11,250 lbs,Texas-Austin,USA
...,...,...,...,...,...,...,...,...
499,ThaddeusYoung,SAS,30.0,F,6-8,235 lbs,Georgia Tech,USA
500,TraeYoung,ATL,11.0,G,6-1,164 lbs,Oklahoma,USA
501,OmerYurtseven,MIA,77.0,C,6-11,275 lbs,Georgetown,Turkey
502,CodyZeller,POR,40.0,F-C,6-11,240 lbs,Indiana,USA


In [5]:
#in case I delete it unfortunately
player_game_attributes = ['PLAYER', 'MIN', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-']


In [6]:
def get_game_stats(html):
    
    """
    Args: html from driver.page_source.encode('utf-8')
    
    Return: A list in which each element correspond to player statistics during the game
    
    """
    soup = BeautifulSoup(html)
    teams_box = soup.find_all('section', 'mb-4 overflow-hidden bg-white shadow-block md:mb-6 md:rounded')
    a = []
    for team_box in teams_box:
        team_name = team_box.find('span', 'block').text
        player_stats = team_box.find('tbody')
        players = player_stats.find_all('tr')
        for i, player in enumerate(players): 
            b = [team_name]
            stats = player.find_all('td')
            #if we are at the row TOTALS of the boxscore we don't care
            if not len(player.find_all('span', 'block py-2 text-xs leading-none')) > 0:
                for i, stat in enumerate(stats):
                    if i == 0:
                        #the first element of the table is the player if the picture, the name and the position
                        #get the text of it give a non sense string

                        #see if there is a more elegant way to deal with
                        player_name = stat.find('span', 'hidden md:inline-block')
                        b.append(player_name.text)
                    else:
                        b.append(stat.text)
            #just put the player name at the beginning
            if len(b) > 1 :
                aux = b[0]
                b[0] = b[1]
                b[1] = aux
                a.append(b)
    return a

def retrieve_games_score(html):
    """
    Args : html of driver.get(URL) follow by driver.page_source.encode of the URL of the game day, 
    maybe we can change it with just a date as input and create the url
    
    Return: the game of this day with the score
    """
    soup = BeautifulSoup(html)
    game_boxes = soup.find_all('div', 'shadow-block bg-white flex md:rounded text-sm relative mb-4')
    games = []
    #if condition check if there was at least one game this day
    if len(game_boxes) > 0:
        for game_box in game_boxes:
            score_left = game_box.find('div', 'w-1/3 text-left').text
            score_right = game_box.find('div', 'w-1/3 text-right').text
            #the order of the team will be left right according to the website
            teams = game_box.find_all('span', 'MatchupCardTeamName_teamName__3i23P')
            match = []
            for team in teams :
                match.append(team.text)
            games.append([match[0], score_left, match[1], score_right])
        return games
    
    
def get_game_team_stats(html):
    
    #try to develop it to understand how use it without go throw driver from selenium
    
    """
    Args: html from driver.page_source.encode('utf-8')
    
    """
    soup = BeautifulSoup(html)
    teams_box = soup.find_all('section', 'mb-4 overflow-hidden bg-white shadow-block md:mb-6 md:rounded')
    a = []
    for team_box in teams_box:
        team_name = team_box.find('span', 'block').text
        player_stats = team_box.find('tbody')
        players = player_stats.find_all('tr')
        for i, player in enumerate(players): 
            b = [team_name]
            stats = player.find_all('td')
            #just take the row TOTALS of the game for each team (the only of the table with a tag 'block xxx')
            if len(player.find_all('span', 'block py-2 text-xs leading-none')) > 0:
                for i, stat in enumerate(stats):
                    if i >= 2:
                        b.append(stat.text)
                a.append(b)
    return a

def get_game_stats_filtered(html, df_player):
    res = []
    a = get_game_stats(html)
    for player_stats in a:
        if df_player['Player'][df_player['Player'] == player_stats[0].replace(' ', '')].shape[0] == 1:
            if len(player_stats)> 3: #test if the player has really played the game
                res.append(player_stats)
    return res

In [7]:
def retrieve_box_score_link3(html):
    """
    Args: html of driver.get(URL) follow by driver.page_source.encode of the URL of the game day, 
    maybe we can change it with just a date as input and create the url
    
    Return: list of link to the box score, the len of the list will be the same as the numnber of
    games during the day
    """
    
    #the watchbox button since the 2012 season change everything 
    
    soup = BeautifulSoup(html)
    game_boxes = soup.find_all('div', 'shadow-block bg-white flex md:rounded text-sm relative mb-4')
    links = []
    #if condition check if there was at least one game this day
    if len(game_boxes) > 0:
        for game_box in game_boxes:
            link = game_box.find('a',
                    {"class": 'TabLink_link__2vpIz Anchor_complexLink__2NtkO'})['href']
            #we get a useless link which look like this '/game/mia-vs-bos-0021000001/box-score#box-score'
            #to make it useful we just have to add the string 'https://nba.com'
            link = link.split('?')[0]
            link = link + '/box-score'
            links.append('https://nba.com' + link)
        return links


In [8]:
def get_season_url(season, begin_month, begin_day, end_month, end_day):
    """
    Args: 
        season: string the year when the season started
        begin_month: string month when the season started, month should be '01', '02', ..., '12'
        begin_day: string day of the month when the season started
    Return: list of every URL game date for all day during the regular season
    """
    path = "https://www.nba.com/games?date="
    
    #initialization of months and day in each months
    
    #IF THE SEASON STARTS IN JANUARY THERE IS AN ERROR!!
    months = []
    for k in [10, 11, 12, 1, 2, 3, 4, 5, 6]:
        if k < 10:
            months.append('0' + str(k))
        else:
            if k >= int(begin_month):
                months.append(str(k))
    day_even = []
    for k in range(1, 31):
        if k < 10:
            day_even.append('0' + str(k))
        else:
            day_even.append(str(k))
    day_odd = []
    for k in range(1, 32):
        if k < 10:
            day_odd.append('0' + str(k))
        else:
            day_odd.append(str(k))
    
    link_list = []
    for month in months:
        if month == begin_month :
            for day in day_odd[(int(begin_day) - 1):] :
                date_ = season + '-' + month + '-' + day
                link_list.append(path + date_)
            if month == '12':
                aux = int(season) + 1
                season = str(aux)
        elif month == end_month :
            for day in day_even[:int(end_day)] :
                date_ = season + '-' + month + '-' + day
                link_list.append(path + date_)
            break
        elif month == '02':
            #check is the year is bisextile
            if (int(season) % 4 == 0) and (int(season)% 10 != 0):
                for day in day_odd[:29]:
                    date_ = season + '-' + month + '-' + day
                    link_list.append(path + date_)
            else :
                for day in day_odd[:28]:
                    date_ = season + '-' + month + '-' + day
                    link_list.append(path + date_)      
        elif month in ['01', '03', '05', '07', '08', '10', '12']:
            for day in day_odd :
                date_ = season + '-' + month + '-' + day
                link_list.append(path + date_)
            if month == '12':
                aux = int(season) + 1
                season = str(aux)
        elif month in ['04', '06', '09', '11']:
            for day in day_even:
                date_ = season + '-' + month + '-' + day
                link_list.append(path + date_)

    return link_list 


In [9]:
s10 = get_season_url('2010', '10', '26', '04', '13')
s11 = get_season_url('2011', '12', '25', '04', '26')
s12 = get_season_url('2012', '10', '30', '04', '17')
s13 = get_season_url('2013', '10', '29', '04', '16')
s14 = get_season_url('2014', '10', '28', '04', '15')
s15 = get_season_url('2015', '10', '27', '04', '13')
s16 = get_season_url('2016', '10', '25', '04', '12')
s17 = get_season_url('2017', '10', '17', '04', '11')
s18 = get_season_url('2018', '10', '16', '04', '13')
s20 = get_season_url('2020', '12', '22', '05', '16')
S = [s10, s11, s12, s13, s14, s15, s16, s17, s18, s20]

In [10]:
s13[50:80]

['https://www.nba.com/games?date=2013-12-18',
 'https://www.nba.com/games?date=2013-12-19',
 'https://www.nba.com/games?date=2013-12-20',
 'https://www.nba.com/games?date=2013-12-21',
 'https://www.nba.com/games?date=2013-12-22',
 'https://www.nba.com/games?date=2013-12-23',
 'https://www.nba.com/games?date=2013-12-24',
 'https://www.nba.com/games?date=2013-12-25',
 'https://www.nba.com/games?date=2013-12-26',
 'https://www.nba.com/games?date=2013-12-27',
 'https://www.nba.com/games?date=2013-12-28',
 'https://www.nba.com/games?date=2013-12-29',
 'https://www.nba.com/games?date=2013-12-30',
 'https://www.nba.com/games?date=2013-12-31',
 'https://www.nba.com/games?date=2014-01-01',
 'https://www.nba.com/games?date=2014-01-02',
 'https://www.nba.com/games?date=2014-01-03',
 'https://www.nba.com/games?date=2014-01-04',
 'https://www.nba.com/games?date=2014-01-05',
 'https://www.nba.com/games?date=2014-01-06',
 'https://www.nba.com/games?date=2014-01-07',
 'https://www.nba.com/games?date=2

In [13]:
id_ = 0
a_score = []
a_team = []
a_player = []
for i, url in enumerate(s13):
    driver.get(url)
    time.sleep(5)
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html)
    links = retrieve_box_score_link3(html)
    date = url.split('=')[1]
    scores_list = retrieve_games_score(html)
    print(links)
    if scores_list is not None:
        for score in scores_list:
            score.append(date)
            a_score.append(score)
        for link in links:
            driver.get(link)
            print(link)
            time.sleep(6)
            html = driver.page_source.encode('utf-8')
            a1 = get_game_team_stats(html)
            for i, team_stats in enumerate(a1):
                if i == 0 :
                    team_stats.append('0')
                    team_stats.append(id_)
                    team_stats.append(date)
                else : 
                    team_stats.append('1')
                    team_stats.append(id_)
                    team_stats.append(date)
                a_team.append(team_stats)
            a2 = get_game_stats_filtered(html, df_player)
            for player_stats in a2:
                player_stats.append(id_)
                player_stats.append(date)
                a_player.append(player_stats)
            id_ += 1
print(a_score)
print(a_player)
print(a_team)

['https://nba.com/game/orl-vs-ind-0021300001/box-score', 'https://nba.com/game/chi-vs-mia-0021300002/box-score', 'https://nba.com/game/lac-vs-lal-0021300003/box-score']
https://nba.com/game/orl-vs-ind-0021300001/box-score
https://nba.com/game/chi-vs-mia-0021300002/box-score


KeyboardInterrupt: 

In [14]:
player_game_attributes = ['PLAYER', 'TEAM', 'MIN', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-', 'GAMEID', 'Date']
team_game_attributes = ['TEAM', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-', 'HOME', 'GAMEID', 'Date']
score_attributes = ['Visitor', 'VisitorScore', 'Home', 'HomeScore', 'Date']

In [15]:
a_score = np.array(a_score)
df_score = pd.DataFrame(a_score, columns = score_attributes)
df_score['GAMEID'] = list(df_score.index)
df_score

,Visitor,VisitorScore,Home,HomeScore,Date,GAMEID
0,Magic,87,Pacers,97,2013-10-29,0
1,Bulls,95,Heat,107,2013-10-29,1
2,Clippers,103,Lakers,116,2013-10-29,2


In [16]:
a_player = np.array(a_player)
df_player = pd.DataFrame(a_player, columns = player_game_attributes)
df_player

,PLAYER,TEAM,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,...,REB,AST,STL,BLK,TO,PF,PTS,+/-,GAMEID,Date
0,Maurice Harkless,Orlando Magic,23:30,6,13,46.2,2,2,100,0,...,1,0,1,0,0,4,14,-3,0,2013-10-29
1,Nikola Vucevic,Orlando Magic,30:39,4,11,36.4,0,0,0.0,0,...,10,3,2,1,5,4,8,-22,0,2013-10-29
2,Victor Oladipo,Orlando Magic,22:35,4,11,36.4,1,2,50.0,3,...,3,2,1,0,3,2,12,-11,0,2013-10-29
3,E'Twaun Moore,Orlando Magic,25:45,3,6,50.0,0,1,0.0,0,...,1,1,2,0,2,2,6,3,0,2013-10-29
4,Paul George,Indiana Pacers,36:05,8,16,50.0,3,6,50.0,5,...,6,5,1,3,4,0,24,11,0,2013-10-29
5,George Hill,Indiana Pacers,29:04,2,8,25.0,0,2,0.0,3,...,2,0,0,0,1,0,7,5,0,2013-10-29
6,Solomon Hill,Indiana Pacers,8:07,0,1,0.0,0,0,0.0,0,...,1,0,0,0,1,1,0,-10,0,2013-10-29
7,Jimmy Butler,Chicago Bulls,29:55,6,12,50.0,2,4,50.0,6,...,3,3,5,1,2,4,20,10,1,2013-10-29
8,Derrick Rose,Chicago Bulls,34:21,4,15,26.7,1,7,14.3,3,...,1,4,0,0,5,2,12,-9,1,2013-10-29
9,Taj Gibson,Chicago Bulls,29:19,4,8,50.0,0,0,0.0,2,...,8,3,0,0,0,1,10,-13,1,2013-10-29


In [17]:
a_team = np.array(a_team)
df_team = pd.DataFrame(a_team, columns = team_game_attributes)
df_team

,TEAM,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,...,AST,STL,BLK,TO,PF,PTS,+/-,HOME,GAMEID,Date
0,Orlando Magic,36,93,38.7,9,19,47.4,6,10,60.0,...,17,10,6,17,26,87,-10,0,0,2013-10-29
1,Indiana Pacers,34,71,47.9,7,17,41.2,22,32,68.8,...,17,4,18,20,13,97,10,1,0,2013-10-29
2,Chicago Bulls,35,83,42.2,7,26,26.9,18,23,78.3,...,23,11,4,18,27,95,-12,0,1,2013-10-29
3,Miami Heat,37,72,51.4,11,20,55.0,22,29,75.9,...,26,10,7,18,21,107,12,1,1,2013-10-29


In [18]:
df_score.to_csv('score_2013-14.csv')
df_player.to_csv('player_2013-14.csv')
df_team.to_csv('team_2013-14.csv')